# Getting started with the Data API

### **Let's search & download some imagery of farmland near Stockton, CA. Here are the steps we'll follow:**

1. Define an Area of Interest (AOI)
2. Save our AOI's coordinates to GeoJSON format
3. Create a few search filters
4. Search for imagery using those filters
5. Activate an image for downloading
6. Download an image

### Requirements
- Python 2.7 or 3+
- requests
- A [Planet API Key](https://www.planet.com/account/#/)

## Define an Area of Interest

An **Area of Interest** (or *AOI*) is how we define the geographic "window" out of which we want to get data.

For the Data API, this could be a simple bounding box with four corners, or a more complex shape, as long as the definition is in [GeoJSON](http://geojson.org/) format. 

For this example, let's just use a simple box. To make it easy, I'll use [geojson.io](http://geojson.io/) to quickly draw a shape & generate GeoJSON output for our box:

![geojsonio.png](images/geojsonio.png)

We only need the "geometry" object for our Data API request:

In [1]:
#40.85219476715068, -96.61594635581866 ####40.85914603929872, -96.59595708657129
# Stockton, CA bounding box (created via geojson.io) 

geojson_geometry = {
  "type": "Polygon",
  "coordinates": [
          [
            [-93.6926474,41.9869467],
            [-93.6905921,41.9869545],
            [-93.6906233,41.9844974],
            [-93.6927761,41.9845267],
            [-93.6926474,41.9869467]
          ]
        ]
}

## Create Filters

Now let's set up some **filters** to further constrain our Data API search:

In [11]:
# get images that overlap with our AOI 

geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2021-06-01T00:00:00.000Z",
    "lte": "2021-09-30T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.1
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

## Searching: Items and Assets

Planet's products are categorized as **items** and **assets**: an item is a single picture taken by a satellite at a certain time. Items have multiple asset types including the image in different formats, along with supporting metadata files.

For this demonstration, let's get a satellite image that is best suited for analytic applications; i.e., a 4-band image with spectral data for Red, Green, Blue and Near-infrared values. To get the image we want, we will specify an item type of `PSScene`, and asset type `ps4b_analytic` (to get a PSScene4Band Analytic asset).

You can learn more about item & asset types in Planet's Data API [here](https://developers.planet.com/docs/apis/data/items-assets/).

Now let's search for all the items that match our filters:

In [19]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth
import time
import pathlib

# API Key stored as an env variable
PLANET_API_KEY = 'PLAK6e194c14d1884f61befe389549a0252f'


item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

print(json.dumps(search_result.json(), indent=1))

{
 "_links": {
  "_first": "https://api.planet.com/data/v1/searches/4be70d4b733f4bc1a7ac1dbea5c496e8/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D",
  "_next": "https://api.planet.com/data/v1/searches/4be70d4b733f4bc1a7ac1dbea5c496e8/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIxLTA2LTAyVDAzOjIyOjU0LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMjEwNjAxXzE2MTkzOV85N18yNDQ0IiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319",
  "_self": "https://api.planet.com/data/v1/searches/4be70d4b733f4bc1a7ac1dbea5c496e8/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D"
 },
 "features": [
  {
   "_links": {
 

Our search returns metadata for all of the images within our AOI that match our date range and cloud coverage filters. It looks like there are multiple images here; let's extract a list of just those image IDs:

In [23]:
def download_order(order_url, img_id, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)
 
    response = r.json()
    deadcount = 0
#     while(response['basic_analytic_4b']['status'] == "inactive"):
#         deadcount +=1
    print("Reponse is:",response['ortho_analytic_8b']['status'])

    results = response['ortho_analytic_8b']['_links']
    #links = result.json()[u"basic_analytic_4b"]["_links"]
    self_link = results["_self"]
    activation_link = results["activate"]
    
    # Request activation of the 'analytic' asset:
    activate_result = \
      requests.get(
        activation_link,
        auth=HTTPBasicAuth(PLANET_API_KEY, '')
      )
    start = time.time()
    activation_status_result = \
      requests.get(
        self_link,
        auth=HTTPBasicAuth(PLANET_API_KEY, '')
      )
    
    while(activation_status_result.json()["status"] == "activating"):
        end = time.time()
        elapse = end - start
        if elapse == 100:
            print('returning due to timeout')
            return False
    print("Asset is:",activation_status_result.json()["status"])
    path = pathlib.Path(os.path.join('data', img_id))
    url = activation_status_result.json()["location"]
    print(path)
    print('{} items to download'.format(len(url)))
    
    if overwrite or not path.exists():
        print('downloading {} to {}'.format(img_id, path))
        r = requests.get(url, allow_redirects=True)
        path.parent.mkdir(parents=True, exist_ok=True)
        open(path, 'wb').write(r.content)
    else:
        print('{} already exists, skipping {}'.format(path, img_id))
            
    return True

In [24]:
# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(len(image_ids))
#print(image_ids)

148


Since we just want a single image, and this is only a demonstration, for our purposes here we can arbitrarily select the first image in that list. Let's do that, and get the `asset` list available for that image:

In [25]:
# For demo purposes, just grab the first image ID
for cnt in range(len(image_ids)):
    id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, image_ids[cnt])
    completed = download_order(id0_url,image_ids[cnt],auth=HTTPBasicAuth(PLANET_API_KEY, ''))
    if completed == True:
        print("downloaded:", cnt)
    # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
#     result = \
#       requests.get(
#         id0_url,
#         auth=HTTPBasicAuth(PLANET_API_KEY, '')
#       )

#     # List of asset types available for this particular satellite image
#     print(result.json().keys())
#     print(result.json()['basic_analytic_4b']['status'])
#     # Parse out useful links
#     links = result.json()[u"basic_analytic_4b"]["_links"]
#     self_link = links["_self"]
#     activation_link = links["activate"]

#     # Request activation of the 'analytic' asset:
#     activate_result = \
#       requests.get(
#         activation_link,
#         auth=HTTPBasicAuth(PLANET_API_KEY, '')
#       )
#     activation_status_result = \
#       requests.get(
#         self_link,
#         auth=HTTPBasicAuth(PLANET_API_KEY, '')
#       )

#     print(activation_status_result.json()["status"])
    

<Response [200]>
Reponse is: activating


KeyboardInterrupt: 

 ## Activation and Downloading
 
The Data API does not pre-generate assets, so they are not always immediately availiable to download. In order to download an asset, we first have to **activate** it.

Remember, earlier we decided we wanted a color-corrected image best suited for *analytic* applications. We can check the status of the PSScene 4-Band analytic asset we want to download like so:
 

In [140]:
# This is "inactive" if the "analytic" asset has not yet been activated; otherwise 'active'
print(result.json()['basic_analytic_4b']['status'])

inactive


Let's now go ahead and **activate** that asset for download:

In [141]:
# Parse out useful links
links = result.json()[u"basic_analytic_4b"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

# Request activation of the 'analytic' asset:
activate_result = \
  requests.get(
    activation_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )

At this point, we wait for the activation status for the asset we are requesting to change from `inactive` to `active`. We can monitor this by polling the "status" of the asset:

In [142]:
activation_status_result = \
  requests.get(
    self_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )
    
print(activation_status_result.json()["status"])

activating


Once the asset has finished activating (status is "active"), we can download it. 

*Note: the download link on an active asset is temporary*

In [143]:
# Image can be downloaded by making a GET with your Planet API key, from here:
download_link = activation_status_result.json()["location"]
print(download_link)

KeyError: 'location'

![stockton_thumb.png](images/stockton_thumb.png)